In [ ]:
import pandas as pd
import time
import torch

## Загрузка данных

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os

In [ ]:
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/DOM')

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DOM/data.csv')
df.head()

,Unnamed: 0,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,0,10368,35,29,"Lisa Simpson: Maggie, look. What's that?",235000,True,9,5.0,Lisa Simpson,Simpson Home,"Maggie, look. What's that?",maggie look whats that,4.0
1,1,10369,35,30,Lisa Simpson: Lee-mur. Lee-mur.,237000,True,9,5.0,Lisa Simpson,Simpson Home,Lee-mur. Lee-mur.,lee-mur lee-mur,2.0
2,2,10370,35,31,Lisa Simpson: Zee-boo. Zee-boo.,239000,True,9,5.0,Lisa Simpson,Simpson Home,Zee-boo. Zee-boo.,zee-boo zee-boo,2.0
3,3,10372,35,33,Lisa Simpson: I'm trying to teach Maggie that ...,245000,True,9,5.0,Lisa Simpson,Simpson Home,I'm trying to teach Maggie that nature doesn't...,im trying to teach maggie that nature doesnt e...,24.0
4,4,10374,35,35,"Lisa Simpson: It's like an ox, only it has a h...",254000,True,9,5.0,Lisa Simpson,Simpson Home,"It's like an ox, only it has a hump and a dewl...",its like an ox only it has a hump and a dewlap...,18.0


In [ ]:
phrases = df['normalized_text'].tolist()
phrases[:10]

['maggie look whats that',
 'lee-mur lee-mur',
 'zee-boo zee-boo',
 'im trying to teach maggie that nature doesnt end with the barnyard i want her to have all the advantages that i didnt have',
 'its like an ox only it has a hump and a dewlap hump and dew-lap hump and dew-lap',
 'you know his blood type how romantic',
 'oh yeah whats my shoe size',
 'ring',
 'yes dad',
 'ooh look maggie what is that do-dec-ah-edron dodecahedron']

In [ ]:
text = [[c for c in ph] for ph in phrases if type(ph) is str]

In [ ]:
text

## Делаем массив с данными

In [ ]:
CHARS = set('abcdefghijklmnopqrstuvwxyz ')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [ ]:
MAX_LEN = 50
X = torch.zeros((len(text), MAX_LEN), dtype=int)
for i in range(len(text)):
    for j, w in enumerate(text[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [ ]:
X

tensor([[10, 17,  7,  ...,  0,  0,  0],
        [25, 24, 24,  ...,  0,  0,  0],
        [16, 24, 24,  ...,  0,  0,  0],
        ...,
        [12, 18,  5,  ...,  0,  0,  0],
        [10, 17,  6,  ...,  1, 25, 24],
        [17, 12,  1,  ...,  0,  0,  0]])

In [ ]:
X[0:10]

tensor([[10, 17,  7,  7, 14, 24,  1, 25, 18, 18,  2,  1,  5, 12, 17,  8, 19,  1,
          8, 12, 17,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [25, 24, 24,  0, 10,  3,  9,  1, 25, 24, 24,  0, 10,  3,  9,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [16, 24, 24,  0, 15, 18, 18,  1, 16, 24, 24,  0, 15, 18, 18,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [14, 10,  1,  8,  9,  6, 14, 20,  7,  1,  8, 18,  1,  8, 24, 17, 27, 12,
          1, 10, 17,  7,  7, 14, 24,  1,  8, 12, 17,  8,  1, 20, 17,  8,  3,  9,
         24,  1,  4, 18, 24, 19, 20,  8,  1, 24, 20,  4,  1,  5],
        [14,  8, 19,  1, 25, 14,  2, 24,  1, 17, 20,  1, 18, 22,  1, 18, 20, 25,
       

## Смотрим на Embedding и RNN ячейку

In [ ]:
embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 28)
t = embeddings(X[0:10])
t

tensor([[[-0.8222, -0.1234,  2.4529,  ...,  1.6618,  0.6998,  0.5379],
         [-0.5747,  0.1099, -1.4984,  ...,  0.0715,  0.0418,  0.9572],
         [-0.4877,  0.4031, -0.6675,  ...,  0.1007, -0.3329, -0.3449],
         ...,
         [-1.4643,  0.2894,  0.4800,  ...,  0.4564, -0.7993, -0.9643],
         [-1.4643,  0.2894,  0.4800,  ...,  0.4564, -0.7993, -0.9643],
         [-1.4643,  0.2894,  0.4800,  ...,  0.4564, -0.7993, -0.9643]],

        [[-0.6488, -2.0182,  0.3656,  ...,  0.0942,  0.1013, -0.7067],
         [ 0.9187, -1.1078,  1.1648,  ..., -1.6011, -0.4873,  0.7884],
         [ 0.9187, -1.1078,  1.1648,  ..., -1.6011, -0.4873,  0.7884],
         ...,
         [-1.4643,  0.2894,  0.4800,  ...,  0.4564, -0.7993, -0.9643],
         [-1.4643,  0.2894,  0.4800,  ...,  0.4564, -0.7993, -0.9643],
         [-1.4643,  0.2894,  0.4800,  ...,  0.4564, -0.7993, -0.9643]],

        [[-0.4694,  0.5889, -0.0547,  ..., -0.2474, -0.1614, -0.0670],
         [ 0.9187, -1.1078,  1.1648,  ..., -1

In [ ]:
t.shape, X[0:10].shape

(torch.Size([10, 50, 28]), torch.Size([10, 50]))

In [ ]:
rnn = torch.nn.RNN(28, 128, batch_first=True)
o, s = rnn(t)
o.shape, s.shape

(torch.Size([10, 50, 128]), torch.Size([1, 10, 128]))

In [ ]:
o, s2 = rnn(t, s)
o.shape, s2.shape

(torch.Size([10, 50, 128]), torch.Size([1, 10, 128]))

## Практика. Реализуйте код модели нейронной сети
3 слоя - embeding (28), скрытая ячейка (128), полносвязанный из состояния rnn в букву (28)

In [ ]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        ## Здесь создать слои

    def forward(self, sentences, state=None):
        ## Здесь применить

In [ ]:
model = Network()

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.05)

In [ ]:
for ep in range(10):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers, _ = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 2.759, Train loss: 1.718
Epoch 1. Time: 2.738, Train loss: 1.680
Epoch 2. Time: 2.721, Train loss: 1.651
Epoch 3. Time: 2.733, Train loss: 1.628
Epoch 4. Time: 2.942, Train loss: 1.609
Epoch 5. Time: 2.897, Train loss: 1.593
Epoch 6. Time: 3.366, Train loss: 1.578
Epoch 7. Time: 3.656, Train loss: 1.564
Epoch 8. Time: 2.883, Train loss: 1.552
Epoch 9. Time: 2.974, Train loss: 1.540


## Практика. Реализуйте код генерации следующей буквы на основе модели
Логика такая:
    - Сначала кормим в нее буквы из sentence (прогревая состояние)
    - Затем пока не получим none (0) берем самую вероятную букву и добавляем ее в sentence
    - Повторяем

In [ ]:
def generate_sentence():
    sentence = ['h', 'e', 'l', 'l', 'o']
    # Todo

In [ ]:
generate_sentence()

helloe l   toaaohutd     toooohun     toooohun     tooo


In [ ]:
for ep in range(10):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers, _ = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    generate_sentence()

Epoch 0. Time: 2.762, Train loss: 1.530
helloe l   toaaohutd     toooohun     toooohun     tooo
Epoch 1. Time: 2.736, Train loss: 1.519
helloe l   toaaohutd     toooohun     toooohun     tooo
Epoch 2. Time: 2.729, Train loss: 1.509
helloe l n toooah n   toaoooutn     toooohun     tooooh
Epoch 3. Time: 2.761, Train loss: 1.500
helloe l n toooah n   toaoooutn     toooohun     tooooh
Epoch 4. Time: 2.746, Train loss: 1.491
helloe l n toooth n  etoaoahnnn     toooohun     tooooh
Epoch 5. Time: 2.932, Train loss: 1.483
helloe l n toooth n  etoaoahnnn     toooohun     tooooh
Epoch 6. Time: 2.804, Train loss: 1.475
helloe l n toooth n  etoaoahnnn     tooooounn     toooo
Epoch 7. Time: 2.790, Train loss: 1.467
helloe l n toooth n  etoaoahnnn     tooooounn     toooo
Epoch 8. Time: 3.052, Train loss: 1.459
helloe l n toooth n  etoaoahnnn     tooooornn     toooo
Epoch 9. Time: 3.189, Train loss: 1.452
helloe l n toooth nn eto tr ntheto e  ntraooaennnn     
